In [2]:
import requests
import time

In [4]:
url_start = 'https://www.basketball-reference.com/awards/awards_{}.html'
years = list(range(1991, 2023))
for year in years:
    url = url_start.format(year)
    data= requests.get(url)
    time.sleep(15)
    with open("mvp/{}.html".format(year), "w+", encoding = 'utf-8') as f:
        f.write(data.text)

In [6]:
from bs4 import BeautifulSoup

In [7]:
with open("mvp/1991.html") as f:
    page = f.read()

In [8]:
soup = BeautifulSoup(page, "html.parser")

In [9]:
soup.find("tr", class_ = "over_header").decompose() # decompose allows to remove

In [11]:
# only want mvp table
# find the sepcific table that we want. table has a specific id mvp, 
# id in html is a globally unique property that only one element usually has
# find the id in the html file which is called <table class ... = id= "mvp"

mvp_table = soup.find(id = "mvp")
# by default is not a string

In [13]:
import pandas as pd
# pandas can read html tables

In [84]:
# 1991 TABLE
mvp_1991 = pd.read_html(str(mvp_table))[0] # need to make mvp_table into a string 
# mvp_1991 
# this is not a dataframe but a list of dataframes so we need to take first element


In [26]:
dfs = []
for year in years:
    with open("mvp/{}.html".format(year)) as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find("tr", class_ = "over_header").decompose() # decompose allows to remove
    mvp_table = soup.find(id = "mvp")
    mvp = pd.read_html(str(mvp_table))[0]
    mvp["Year"] = year # need to see what year the voting occured 
    
    dfs.append(mvp)


In [32]:
mvps_combined = pd.concat(dfs) # concat allows us to combine all the dfs
mvps_combined.head()

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,1,Michael Jordan,27,CHI,77.0,891.0,960,0.928,82,37.0,...,6.0,5.5,2.7,1.0,0.539,0.312,0.851,20.3,0.321,1991
1,2,Magic Johnson,31,LAL,10.0,497.0,960,0.518,79,37.1,...,7.0,12.5,1.3,0.2,0.477,0.320,0.906,15.4,0.251,1991
2,3,David Robinson,25,SAS,6.0,476.0,960,0.496,82,37.7,...,13.0,2.5,1.5,3.9,0.552,0.143,0.762,17.0,0.264,1991
3,4,Charles Barkley,27,PHI,2.0,222.0,960,0.231,67,37.3,...,10.1,4.2,1.6,0.5,0.570,0.284,0.722,13.4,0.258,1991
4,5,Karl Malone,27,UTA,0.0,142.0,960,0.148,82,40.3,...,11.8,3.3,1.1,1.0,0.527,0.286,0.770,15.5,0.225,1991


In [29]:
mvps_combined.to_csv("mvps.csv")

In [30]:
# NEED all player data

In [34]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

In [35]:
url = player_stats_url.format(1991)
data = requests.get(url)
with open("player/1991.html", "w+", encoding = 'utf-8') as f:
        f.write(data.text)
# doesnt work 
# a lot of websites have javascript which you need to run on a website

# PLAYER STATS NOT FROM BASKETBALL REFERENCE

In [51]:
url = 'https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=PerGame&Scope=S&Season=2022-23&SeasonType=Regular%20Season&StatCategory=PTS'

In [55]:
r = requests.get(url).json()


In [58]:
table_headers = r["resultSet"]["headers"]

In [76]:
df = pd.DataFrame(r["resultSet"]["rowSet"], columns = table_headers)

In [135]:
url = 'https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=PerGame&Scope=S&Season={}-{}&SeasonType=Regular%20Season&StatCategory=PTS'
df_ha = []
for year in years:
    if year == 2022:
        r = requests.get(url.format(year, 23)).json()
        table_headers = r["resultSet"]["headers"]
        a = pd.DataFrame(r["resultSet"]["rowSet"], columns = table_headers)
        a["Year"] = year
        df_ha.append(a)
        
    r = requests.get(url.format(year, (year + 1) % 100)).json()
    table_headers = r["resultSet"]["headers"]
    a = pd.DataFrame(r["resultSet"]["rowSet"], columns = table_headers)
    a["Year"] = year
    df_ha.append(a)

    

In [145]:
r = requests.get('https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=PerGame&Scope=S&Season=2008-09&SeasonType=Regular%20Season&StatCategory=PTS').json()
table_headers = r["resultSet"]["headers"]
a = pd.DataFrame(r["resultSet"]["rowSet"], columns = table_headers)
a["Year"] = 2008
df_ha.append(a)


In [165]:
len(df_ha)

43

In [166]:
abra = pd.concat(df_ha)

In [167]:
abra.to_csv("players1.csv")

## Team records

In [95]:
teams_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

In [96]:
'''
for year in years:
    url = teams_url.format(year)
    data= requests.get(url)
    time.sleep(15)
    with open("team/{}.html".format(year), "w+", encoding = 'utf-8') as f:
        f.write(data.text) '''

In [105]:
dfs_team_standings = []
for year in years:
    with open("team/{}.html".format(year)) as f:
        page = f.read()
        
    soup = BeautifulSoup(page, "html.parser")
    soup.find("tr", class_ = "thead").decompose() # decompose allows to remove
    team_table = soup.find(id = "divs_standings_E")
    team = pd.read_html(str(team_table))[0]
    team["Year"] = year
    team["Team"] = team["Eastern Conference"]
    del team["Eastern Conference"]
    dfs_team_standings.append(team)
    

    soup = BeautifulSoup(page, "html.parser")
    soup.find("tr", class_ = "thead").decompose() # decompose allows to remove
    team_table = soup.find(id = "divs_standings_W")
    team = pd.read_html(str(team_table))[0]
    team["Year"] = year
    team["Team"] = team["Western Conference"]
    del team["Western Conference"]
    dfs_team_standings.append(team)

In [106]:
standings = pd.concat(dfs_team_standings)

In [107]:
standings

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics*
1,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers*
2,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks*
3,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets
...,...,...,...,...,...,...,...,...,...
13,56,26,.683,—,115.6,109.9,5.37,2022,Memphis Grizzlies*
14,52,30,.634,4.0,108.0,104.7,3.12,2022,Dallas Mavericks*
15,36,46,.439,20.0,109.3,110.3,-0.84,2022,New Orleans Pelicans*
16,34,48,.415,22.0,113.2,113.0,0.02,2022,San Antonio Spurs


In [108]:
standings.to_csv("standings.csv")